In [ ]:
#import libraries
import numpy as np
from ase.io import read, write
from ase import Atoms
import os
os.makedirs('config', exist_ok=True)
import torch

In [ ]:
#importing the full data (this is in case I have like the whole data in one file)
#db = read(. ':')

In [ ]:
#splitting the data into the training and test sets

In [ ]:
%%writefile config_new_model.yml
model: ""
num_channels: 32
max_L: 0
r_max: 4.0
name: ""
train_file: ".xyz"
valid_fraction: 0.1
test_file: ".xyz"
energy_key: "energy"
forces_key: "forces"
batch_size: 20
max_num_epochs: 100
device: cpu
seed: 123

In [ ]:
#Perform training
import warnings
warnings.filterwarnings('ignore')
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

#defining the training function
def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ['program', '--config', config_file_path]
    mace_run_train_main()

#calling the function
train_mace('yml') #aca va el nombre del yml file 

In [ ]:
#once the training is done, then a calculation can be performed using ase
